In [24]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    content = list(islice(myfile, 150000))

In [25]:
print(content[:9])

['Néstor_Gabriel_Martinena néstor_gabriel_martinena NP00000 0\n', '( ( Fpa 0\n', 'La el DA0FS0 0\n', 'Plata plata NP00000 10501473\n', ', , Fc 0\n', 'Buenos_Aires buenos_aires NP00000 0\n', ', , Fc 0\n', 'Argentina argentina NP00000 02035234\n', ', , Fc 0\n']


In [26]:
sent = []
for s in content:
    sent.append(s.split())

In [27]:
sent[:5]

[['Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'],
 ['(', '(', 'Fpa', '0'],
 ['La', 'el', 'DA0FS0', '0'],
 ['Plata', 'plata', 'NP00000', '10501473'],
 [',', ',', 'Fc', '0']]

In [28]:
#Creamos una tripla del estilo (lemma, PoS tag, synset)
words = []
#sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [29]:
#Creamos un diccionario con las palabras y su cantidad de apariciones
count = dict()
for fst,snd,trd in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [30]:
count['entrar']

35

In [31]:
words[:4]

[('néstor_gabriel_martinena', 'NP00000', '0'),
 ('el', 'DA0FS0', '0'),
 ('plata', 'NP00000', '10501473'),
 ('buenos_aires', 'NP00000', '0')]

In [32]:
#Check es una lista de listas de tuplas del estilo (lemma,PoS tag, synset), así obtenemos una separación por oración.
aux = []
check = []
for fst,snd,trd in words:
    aux.append((fst,snd, trd))
    if fst == '.':
        check.append(aux)
        aux = []

In [57]:
check[:2]

[[('néstor_gabriel_martinena', 'NP00000', '0'),
  ('el', 'DA0FS0', '0'),
  ('plata', 'NP00000', '10501473'),
  ('buenos_aires', 'NP00000', '0'),
  ('argentina', 'NP00000', '02035234'),
  ('ser', 'VSIP3S0', '01775973'),
  ('uno', 'DI0MS0', '0'),
  ('futbolista', 'NCCS000', '0'),
  ('argentino', 'AQ0MS0', '00278090'),
  ('.', 'Fp', '0')],
 [('jugar', 'VMM02S0', '00727813'),
  ('de', 'SPS00', '0'),
  ('delantero', 'NCMS000', '00466114'),
  ('y', 'CC', '0'),
  ('su', 'DP3CS0', '0'),
  ('equipo', 'NCMS000', '06093198'),
  ('actual', 'AQ0CS0', '01667781'),
  ('ser', 'VSIP3S0', '01775973'),
  ('gimnasia', 'NP00000', '00275488'),
  ('y', 'CC', '0'),
  ('esgrima_la_plata_de_la_primera_división_de_argentina', 'NP00000', '0'),
  ('.', 'Fp', '0')]]

In [33]:
from collections import defaultdict

In [34]:
#Diccionario con todas las palabras anteriores a una palabra (ventana de 1) --> dicc_backward
#Diccionario con todas las palabras siguientes a una palabra (ventana de 1) --> dicc_forward
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[T.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]'))

In [35]:
#Creación del diccionario de features
#Recordemos que words es una lista de tuplas del tipo (lemma,Pos,synset)
dicc = dict()
vec = dict()    #vector de clases
for fst,snd,trd in words:
    features = defaultdict(int)
    pos = 'PoS__' + snd
    #La palabra se encuentra en el dict
    if fst not in dicc and fst not in {'.', ',', ':', ';'} and fst.isalpha() and count[fst] > 1 and trd != 0:
        dicc[fst] = features
        vec[fst] = trd
        features[pos] = 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[1]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[1]+"+1"] += 1
    #La palabra no se encuentra en el dict     
    elif fst in dicc:
        if trd not in vec[fst]:
            dicc[fst] = features
            vec[fst] = trd
            for i in dicc_backward[fst]:
                features[i[0]+"-1"] += 1
                features[i[1]+"-1"] += 1
            for i in dicc_forward[fst]:
                features[i[0]+"+1"] += 1
                features[i[1]+"+1"] += 1
        has_it = dicc[fst]
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1

In [36]:
#Controlamos que el largo sea consistente
print(len(dicc), len(vec))

5524 5524


In [37]:
#Para remover stopwords
from nltk.corpus import stopwords

In [38]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    if key not in stopwords.words('spanish'):
        final_check.append(key)
        matrix.append(value)
for key, value in vec.items():       #vector de synsets
    if key not in stopwords.words('spanish'):
        vector.append(int(value))

In [39]:
#Transformamos nuestra lista de diccionarios de features en un vector
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [40]:
X

<5445x13727 sparse matrix of type '<class 'numpy.float64'>'
	with 113085 stored elements in Compressed Sparse Row format>

In [41]:
mejor_k = X.shape

In [42]:
#seleccionamos el 70% de los features
k_best = mejor_k[1]*0.7

In [43]:
##########################################Supervisada, clase: synset ##################################################

In [44]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [45]:
#Normalización
import sklearn
X = sklearn.preprocessing.normalize(X)

In [46]:
#Método supervisado para feature selection usando la función chi2
Y = SelectKBest(chi2, k=int(k_best)).fit_transform(X, vector)

In [71]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=90)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=90, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [72]:
labels = km.predict(Y)

In [73]:
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(final_check[i])
print(clusters)

defaultdict(<class 'set'>, {40: {'nbc', 'plata', 'constitución', 'santo', 'francisca', 'nasa', 'ama', 'estación', 'bujía', 'conmebol', 'cart', 'época', 'compañía', 'ecu', 'belga', 'alhambra', 'academia', 'saga', 'revolución', 'oea', 'organización', 'mancha', 'minigun', 'nba', 'naturaleza', 'ncaa', 'mode', 'wnba', 'cosecha', 'primera', 'cba', 'crisis', 'acadia', 'fraternidad', 'patagonia', 'marmota', 'contratierra', 'india', 'bestia', 'vertiente'}, 73: {'pamplona', 'shorty', 'floride', 'sirius', 'kubuntu', 'kirtland', 'kansas', 'riga', 'canadá', 'song', 'rhyno', 'torrevieja', 'corrientes', 'internet', 'sevilla', 'bdfa', 'mendoza', 'jesse', 'japón', 'hawaii', 'munich', 'indochina', 'hungría', 'smith', 'pylea', 'suiza', 'michigan', 'argentina', 'angel', 'cordelia', 'norteamérica', 'salta', 'groove', 'durango', 'méxico', 'múnich', 'wimbledon', 'antioquia', 'altair', 'cienfuegos', 'cross', 'alemania', 'catamarca', 'francia', 'luxemburgo', 'praga', 'tokio', 'neuquén', 'valenzuela', 'santiago

In [79]:
print(clusters[11])

{'granada', 'manchester', 'céspedes', 'parís', 'downingtown', 'cayo', 'putumayo', 'tlalpan', 'tlaltizapán', 'stuttgart', 'utc', 'viena', 'manila', 'vancouver', 'jacksonville', 'alabama', 'bacadéhuachi', 'haarlem', 'newcastle', 'glasgow', 'copenhague', 'gama', 'reinosa', 'álamos', 'zacatecas', 'bethesda', 'chicago', 'leeds', 'bristol', 'asturias', 'ibanez', 'londres', 'hamburgo', 'berlin', 'palmyra', 'estocolmo', 'oxford', 'envigado'}


In [74]:
print(clusters[56])

{'nuevamente', 'brillantemente', 'completamente', 'absolutamente', 'bastante', 'sumamente', 'frecuentemente', 'solamente', 'empty', 'biológicamente', 'automáticamente', 'dramáticamente', 'naturalmente', 'obviamente', 'prácticamente', 'drásticamente', 'esencialmente', 'temporalmente', 'mayoritariamente', 'bien', 'tan', 'usualmente', 'popularmente', 'históricamente', 'flood', 'fácilmente', 'constantemente', 'extremadamente', 'quizás', 'oficialmente', 'altamente', 'aparentemente', 'típicamente', 'totalmente', 'formalmente'}


In [76]:
print(clusters[21])

{'enterrar', 'equipar', 'elaborar', 'preclasificado', 'remover', 'traspasar', 'transferir', 'emigrar', 'decorar', 'obsesionar', 'firmar', 'asignar', 'reconstruir', 'idear', 'editar', 'manchar', 'amurallar', 'prohibir', 'conocer', 'elevar', 'diseñar', 'reclamar', 'comprar', 'concebir', 'armar', 'atrapar', 'rodar', 'apropiar', 'nombrar', 'herir', 'elogiar', 'apasionar', 'acoplar', 'adiestrar', 'condenar', 'contratar', 'sobrevivir', 'encarcelar', 'seleccionar', 'acuñar', 'involucrar', 'reorganizar', 'asustar', 'confirmar', 'solicitar', 'curvar', 'cesar', 'galardonar', 'relacionar', 'presionar', 'inaugurar', 'fusilar', 'filmar', 'lanzar', 'registrar', 'cerrar', 'promulgar', 'conceder', 'poblar', 'esparcir', 'confundir', 'sepultar', 'vestir', 'escoger', 'citar', 'catalogar', 'mezclar', 'crecer', 'emplear'}


In [78]:
print(clusters[37])

{'popular', 'general', 'liberal', 'rural', 'leal', 'conceptual', 'similar', 'formal', 'provincial', 'procedente', 'auxiliar', 'comercial', 'baterista', 'posterior', 'goreana', 'ausente', 'meridional', 'franquista', 'mineral', 'comunista', 'septentrional', 'judicial', 'municipal', 'triangular', 'especial', 'terminal', 'funcional', 'estelar', 'sostenible', 'alfa', 'brillante', 'imposible', 'occidental', 'excepcional', 'ciclista', 'suave', 'constitucional', 'disponible', 'peatonal', 'altisonante', 'fascista', 'ilustre', 'elemental', 'canadiense', 'criminal', 'sample', 'fácil', 'territorial', 'moral', 'laboral', 'ejemplar', 'medieval', 'presidencial'}


In [77]:
print(clusters[2])

{'cabeza', 'pretensión', 'reforma', 'ley', 'leyenda', 'clase', 'pieza', 'orden', 'pendiente', 'pirámide', 'asociación', 'noche', 'frontera', 'cartera', 'línea', 'autenticidad', 'casta', 'tierra', 'actividad', 'expulsión', 'institución', 'presencia', 'superficie', 'serie', 'ironía', 'combinación', 'vida', 'presidencia', 'obligación', 'lengua', 'edición', 'mitad', 'toma', 'constelación', 'estrella', 'cara', 'habilidad', 'situación', 'figura', 'esencia', 'dirección', 'prueba', 'iglesia', 'cuestión', 'lectura', 'producción', 'administración', 'base', 'masa', 'fiesta', 'marca', 'zona', 'pérdida', 'promoción', 'colección', 'favorito', 'práctica', 'hora', 'razón', 'humillación', 'celebración', 'tendencia', 'bolsa', 'necesidad', 'idea', 'carátula', 'semifinal', 'cima', 'donación', 'categoría', 't', 'desembocadura', 'colonia', 'proliferación', 'casa', 'estructura', 'aceptación', 'construcción', 'pintura', 'existencia', 'singularidad', 'izquierda', 'ocupación', 'entrada', 'sección', 'parroquia',

In [63]:
print(clusters[102])

{'enorme', 'consecuente', 'simple', 'implacable', 'frágil', 'alucinante', 'grave', 'doble', 'breve', 'consiguiente', 'impresionante', 'sugerente', 'sublime', 'brutal', 'agradable', 'cuádruple', 'notable', 'gran', 'sutil', 'inusual'}


In [69]:
print(clusters[57])

{'rival', 'humanista', 'esquimal', 'aspirante', 'guionista', 'moralista', 'alienígenas', 'artista', 'not', 'portavoz', 'interfaz', 'congresista', 'novelista', 'habitante', 'punta', 'fabricante', 'piloto', 'atleta', 'ninja', 'but', 'guía', 'jinete', 'mar', 'policía', 'inmigrante', 'estudiante', 'element', 'defensa', 'agente', 'guitarrista', 'tenista', 'testigo', 'azúcar', 'cliente', 'solista', 'fan', 'cantante', 'periodista', 'amante', 'líder', 'futbolista', 'adolescente', 'juez', 'protagonista', 'aprendiz', 'batería'}
